In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Carregamento de dados

In [39]:
def load_dataset(dataset_name, header=None):
    train_path = f'./data/{dataset_name}train.csv'
    test_path = f'./data/{dataset_name}test.csv'

    train_data = pd.read_csv(filepath_or_buffer=train_path, header=header)
    test_data = pd.read_csv(filepath_or_buffer=test_path, header=header)
    
    return train_data, test_data

In [43]:
breast_cancer_train_data, breast_cancer_test_data = load_dataset('breast_cancer_coimbra_', header=0)
print(breast_cancer_train_data.shape, breast_cancer_test_data.shape)

(92, 10) (24, 10)


In [44]:
breast_cancer_train_data.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,44,20.760000,86,7.553,1.600000,14.0900,20.320000,7.64000,63.610,1
1,46,20.830000,88,3.420,0.742368,12.8700,18.550000,13.56000,301.210,2
2,53,36.790166,101,10.175,2.534932,27.1841,20.030000,10.26309,695.754,1
3,54,30.483158,90,5.537,1.229214,12.3310,9.731380,10.19299,1227.910,1
4,34,24.242424,92,21.699,4.924226,16.7353,21.823745,12.06534,481.949,2


### red wine dataset columns
0 - fixed acidity (tartaric acid - g / dm^3)
1 - volatile acidity (acetic acid - g / dm^3)
2 - citric acid (g / dm^3)
3 - residual sugar (g / dm^3)
4 - chlorides (sodium chloride - g / dm^3)
5 - free sulfur dioxide (mg / dm^3)
6 - total sulfur dioxide (mg / dm^3)
7 - density (g / cm^3)
8 - pH
9 - sulphates (potassium sulphate - g / dm3)
10 - alcohol (% by volume)
Output variable (based on sensory data): 
11 - quality (score between 0 and 10)

In [41]:
wine_train_data, wine_test_data = load_dataset('wineRed-', header=None)
print(wine_train_data.shape, wine_test_data.shape)

(1279, 12) (320, 12)


In [42]:
wine_train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,0.640,0.17,5.40,0.168,52.0,98.0,0.99736,3.28,0.50,9.5,5
1,10.4,0.440,0.73,6.55,0.074,38.0,76.0,0.99900,3.17,0.85,12.0,7
2,10.7,0.430,0.39,2.20,0.106,8.0,32.0,0.99860,2.89,0.50,9.6,5
3,8.9,0.635,0.37,1.70,0.263,5.0,62.0,0.99710,3.00,1.09,9.3,5
4,7.8,0.570,0.09,2.30,0.065,34.0,45.0,0.99417,3.46,0.74,12.7,8
